In [2]:
#@title Importing python modules { display-mode: "form" }
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
import os
from scipy.sparse import coo_matrix
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow.keras as K
import sklearn.metrics as skm
import scipy
import sys


from tensorflow.python.keras import activations
from tensorflow.python.keras import backend as Kb
from tensorflow.python.keras import constraints
from tensorflow.python.keras import initializers
from tensorflow.python.keras import regularizers
from tensorflow.python.keras.engine.base_layer import InputSpec
from tensorflow.python.keras.engine.base_layer import Layer
from tensorflow.python.keras.utils import conv_utils
from tensorflow.python.keras.utils import tf_utils
from tensorflow.python.util.tf_export import tf_export

from sklearn.metrics import average_precision_score, roc_auc_score

In [3]:
#@title Utility functions { display-mode: "form" }
#

tf.keras.backend.set_epsilon(0.00001)
global weight_negative_class
global weight_possitive_class
global data_path

def progressBar(value, end_value, bar_length=50):
    percent = float(value + 1) / end_value
    arrow = '-' * int(round(percent * bar_length) - 1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write("\r [{0}] {1}%".format(arrow + spaces, int(round(percent * 100))))

    if value >= end_value:
        sys.stdout.write("\n")

    sys.stdout.flush()


def sensitivity(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_pred = tf.keras.backend.clip(y_pred, 0, 1)
    y_true = tf.keras.backend.clip(y_true, 0, 1)

    y_pred = tf.keras.backend.round(y_pred)

    true_p = K.backend.sum(K.backend.round(y_pred) * y_true)
    pos = tf.keras.backend.sum(y_true)
    sensitivity1 = tf.keras.backend.clip((true_p / (pos + 0.00001)), 0, 1)
    return sensitivity1

def specificity(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    y_pred = tf.keras.backend.clip(y_pred, 0, 1)
    y_true = tf.keras.backend.clip(y_true, 0, 1)

    neg_y_true = 1 - y_true
    neg_y_pred = 1 - K.backend.round(y_pred)
    fp = K.backend.sum(neg_y_true * K.backend.round(y_pred))
    tn = K.backend.sum(neg_y_true * neg_y_pred)
    specificity1 = tn / (tn + fp + 0.00001)
    return tf.keras.backend.clip(specificity1, 0, 1)


def evaluate_performance(y,p):
    print("\n")
    print("Confusion matrix")
    confusion_matrix = skm.confusion_matrix(y, p.round())
    print(confusion_matrix)

    fpr, tpr, thresholds = skm.roc_curve(y, p)
    roc_auc = skm.auc(fpr, tpr)
    print("\n")
    print("Area under the Curve (AUC) = ", roc_auc)

    specificity = confusion_matrix[0, 0] / (confusion_matrix[0, 0] + confusion_matrix[0, 1])
    print('Specificity = ', specificity)

    sensitivity = confusion_matrix[1, 1] / (confusion_matrix[1, 0] + confusion_matrix[1, 1])
    print('Sensitivity = ', sensitivity)
    print("F_1 score = " + str(skm.f1_score(y, p.round())))
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
            lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.plot(1 - specificity, sensitivity, color='b', marker='o')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()



In [ ]:
# Hyperparameters of the simulation and the network training.

jobid = 1
modeltype = "GenNet_proof_of_concept_network"
weight_possitive_class = 1 
weight_negative_class = 1
batch_size = 64
startindex = 0
stopindex = -1
inputsize = 10
max_id = 10
epochs = 100

In [30]:
def GenNet_proof_of_concept_network(inputsize):

    inputs_ = K.Input((inputsize,), name='inputs_')

    Input_layer = K.layers.Reshape(input_shape=(inputsize,), target_shape=(inputsize, 1))(inputs_)
    
    Gene_layer = K.layers.Dense(10, activation='relu')(Input_layer)
    BatchNorm = K.layers.BatchNormalization(name="inter_out")(Gene_layer)
    #Gene_layer = K.layers.Activation("relu", name="gene_activation")(BatchNorm)
    Gene_layer = K.layers.Flatten()(BatchNorm)
    Prediction_layer = K.layers.Dense(units=1, kernel_regularizer=K.regularizers.l1(0.01))(Gene_layer)
    Prediction_layer = K.layers.Activation("sigmoid", name="output_activation")(Prediction_layer)
    print(Prediction_layer.shape)
    model = K.Model(inputs=inputs_, outputs=Prediction_layer)
    return model


In [31]:
performance = pd.read_csv('./performance.csv',index_col=0)
performance_sub = performance[performance.iloc[:, 3] >= performance.iloc[:, 4] ].iloc[:, [0, 3, 4]]

In [33]:
performance_sub

,auroc_ori,auroc_12d,auroc_NN
hibachi-res_BernoulliNB_ExtraTreesClassifier-s37061_0.045.txt,0.646940,0.617401,0.556554
hibachi-res_BernoulliNB_GaussianNB-s23944_0.167.txt,0.492452,0.763084,0.605533
hibachi-res_BernoulliNB_GaussianNB-s2559_0.066.txt,0.735088,0.946115,0.813318
hibachi-res_BernoulliNB_GaussianNB-s43757_0.077.txt,0.480856,0.874736,0.567524
hibachi-res_BernoulliNB_GaussianNB-s78896_0.076.txt,0.760165,0.809078,0.770130
...,...,...,...
hibachi-res_XGBClassifier_MLPClassifier-s29528_0.346.txt,0.483035,0.948203,0.616213
hibachi-res_XGBClassifier_MLPClassifier-s53348_0.349.txt,0.514653,0.878161,0.556247
hibachi-res_XGBClassifier_MLPClassifier-s61608_0.257.txt,0.562025,0.878902,0.643604
hibachi-res_XGBClassifier_MLPClassifier-s71757_0.300.txt,0.514806,0.635754,0.570939


In [ ]:
training_data_path = './rethink-prs-master/mb-mdr/reformatted-data/train/'
test_data_path = './rethink-prs-master/mb-mdr/reformatted-data/train/'
training_data = os.listdir(training_data_path)

for file in performance_sub.index.tolist():

    train = pd.read_csv(training_data_path + file, sep='\t').astype(int)
    SNPs = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']
    xtrain, ytrain = train[SNPs].values, train['Class'].values
    test = pd.read_csv(test_data_path + file, sep='\t').astype(int)

    xtrain, ytrain = train[SNPs].iloc[:600].values, train['Class'].iloc[:600].values
    xval, yval = train[SNPs].iloc[600:].values, train['Class'].iloc[600:].values
    xtest, ytest = test[SNPs].values, test['Class'].values


    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
    
    model = GenNet_proof_of_concept_network(inputsize=int(inputsize))  
    model.compile(loss='binary_crossentropy', optimizer=optimizer, 
                  metrics=["accuracy", sensitivity, specificity]) # compile the network
    earlystop = K.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001,
                                          patience=50, verbose=0, mode='auto',
                                          restore_best_weights=True) # we stop training after not improving after 50 epochs on the validation data
    model.fit(x=xtrain, y=ytrain, batch_size=batch_size, epochs=epochs, 
              verbose=1, callbacks=[earlystop], validation_data=(xval,yval), shuffle=True) # train the neural network
    
    
    ypred = model.predict(xtest)
    auroc = roc_auc_score(ytest, ypred)
    
    performance_sub.loc[file, 'auroc_NN_new'] = auroc

(None, 1)
Epoch 1/100
10/10 [==============================] - 1s 26ms/step - loss: 0.8232 - accuracy: 0.5396 - sensitivity: 0.5675 - specificity: 0.5122 - val_loss: 0.8258 - val_accuracy: 0.5250 - val_sensitivity: 0.0792 - val_specificity: 0.9533
Epoch 2/100
10/10 [==============================] - 0s 5ms/step - loss: 0.8142 - accuracy: 0.5169 - sensitivity: 0.5802 - specificity: 0.4635 - val_loss: 0.8151 - val_accuracy: 0.5000 - val_sensitivity: 0.1104 - val_specificity: 0.7993
Epoch 3/100
10/10 [==============================] - 0s 5ms/step - loss: 0.7909 - accuracy: 0.5659 - sensitivity: 0.6220 - specificity: 0.5065 - val_loss: 0.8057 - val_accuracy: 0.4850 - val_sensitivity: 0.1498 - val_specificity: 0.7380
Epoch 4/100
10/10 [==============================] - 0s 5ms/step - loss: 0.7818 - accuracy: 0.5706 - sensitivity: 0.6236 - specificity: 0.5106 - val_loss: 0.7974 - val_accuracy: 0.4650 - val_sensitivity: 0.2888 - val_specificity: 0.6010
Epoch 5/100
10/10 [======================

10/10 [==============================] - 0s 5ms/step - loss: 0.6517 - accuracy: 0.6338 - sensitivity: 0.4975 - specificity: 0.7706 - val_loss: 0.6764 - val_accuracy: 0.6100 - val_sensitivity: 0.5729 - val_specificity: 0.7008
Epoch 70/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6542 - accuracy: 0.6219 - sensitivity: 0.4851 - specificity: 0.7674 - val_loss: 0.6749 - val_accuracy: 0.6050 - val_sensitivity: 0.5729 - val_specificity: 0.6508
Epoch 71/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6647 - accuracy: 0.6199 - sensitivity: 0.5043 - specificity: 0.7706 - val_loss: 0.6744 - val_accuracy: 0.6050 - val_sensitivity: 0.5729 - val_specificity: 0.6508
Epoch 72/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6559 - accuracy: 0.6355 - sensitivity: 0.5084 - specificity: 0.7683 - val_loss: 0.6739 - val_accuracy: 0.6100 - val_sensitivity: 0.5729 - val_specificity: 0.7008
Epoch 73/100
10/10 [==============================] - 0s 5ms/

10/10 [==============================] - 0s 5ms/step - loss: 0.7110 - accuracy: 0.6237 - sensitivity: 0.5495 - specificity: 0.6964 - val_loss: 0.7482 - val_accuracy: 0.5200 - val_sensitivity: 0.5106 - val_specificity: 0.4685
Epoch 38/100
10/10 [==============================] - 0s 4ms/step - loss: 0.7063 - accuracy: 0.6061 - sensitivity: 0.5637 - specificity: 0.6419 - val_loss: 0.7468 - val_accuracy: 0.5200 - val_sensitivity: 0.5010 - val_specificity: 0.4765
Epoch 39/100
10/10 [==============================] - 0s 5ms/step - loss: 0.7088 - accuracy: 0.6128 - sensitivity: 0.5419 - specificity: 0.6806 - val_loss: 0.7443 - val_accuracy: 0.4900 - val_sensitivity: 0.4599 - val_specificity: 0.4670
Epoch 40/100
10/10 [==============================] - 0s 5ms/step - loss: 0.7111 - accuracy: 0.6037 - sensitivity: 0.5389 - specificity: 0.6731 - val_loss: 0.7430 - val_accuracy: 0.4750 - val_sensitivity: 0.4330 - val_specificity: 0.4670
Epoch 41/100
10/10 [==============================] - 0s 5ms/

10/10 [==============================] - 0s 4ms/step - loss: 0.7489 - accuracy: 0.6296 - sensitivity: 0.6736 - specificity: 0.5875 - val_loss: 0.7815 - val_accuracy: 0.5400 - val_sensitivity: 0.9463 - val_specificity: 0.1834
Epoch 6/100
10/10 [==============================] - 0s 4ms/step - loss: 0.7053 - accuracy: 0.6662 - sensitivity: 0.6846 - specificity: 0.6471 - val_loss: 0.7648 - val_accuracy: 0.5800 - val_sensitivity: 0.8967 - val_specificity: 0.3198
Epoch 7/100
10/10 [==============================] - 0s 5ms/step - loss: 0.7068 - accuracy: 0.6673 - sensitivity: 0.6938 - specificity: 0.6380 - val_loss: 0.7494 - val_accuracy: 0.6150 - val_sensitivity: 0.8638 - val_specificity: 0.4405
Epoch 8/100
10/10 [==============================] - 0s 5ms/step - loss: 0.6820 - accuracy: 0.6760 - sensitivity: 0.6976 - specificity: 0.6551 - val_loss: 0.7351 - val_accuracy: 0.6500 - val_sensitivity: 0.8404 - val_specificity: 0.5163
Epoch 9/100
10/10 [==============================] - 0s 6ms/step

10/10 [==============================] - 0s 5ms/step - loss: 0.5617 - accuracy: 0.7102 - sensitivity: 0.7133 - specificity: 0.7026 - val_loss: 0.5399 - val_accuracy: 0.7550 - val_sensitivity: 0.7857 - val_specificity: 0.7583
Epoch 74/100
10/10 [==============================] - 0s 6ms/step - loss: 0.5517 - accuracy: 0.7288 - sensitivity: 0.7393 - specificity: 0.7218 - val_loss: 0.5391 - val_accuracy: 0.7450 - val_sensitivity: 0.7774 - val_specificity: 0.7510
Epoch 75/100
10/10 [==============================] - 0s 5ms/step - loss: 0.5749 - accuracy: 0.7080 - sensitivity: 0.7253 - specificity: 0.6825 - val_loss: 0.5389 - val_accuracy: 0.7500 - val_sensitivity: 0.7774 - val_specificity: 0.7583
Epoch 76/100
10/10 [==============================] - 0s 5ms/step - loss: 0.5461 - accuracy: 0.7280 - sensitivity: 0.7668 - specificity: 0.6955 - val_loss: 0.5388 - val_accuracy: 0.7450 - val_sensitivity: 0.7774 - val_specificity: 0.7510
Epoch 77/100
10/10 [==============================] - 0s 5ms/

...

10/10 [==============================] - 0s 4ms/step - loss: 0.6534 - accuracy: 0.6561 - sensitivity: 0.6163 - specificity: 0.6930 - val_loss: 0.6709 - val_accuracy: 0.6500 - val_sensitivity: 0.6296 - val_specificity: 0.7077
Epoch 62/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6728 - accuracy: 0.6395 - sensitivity: 0.5918 - specificity: 0.6974 - val_loss: 0.6704 - val_accuracy: 0.6450 - val_sensitivity: 0.6224 - val_specificity: 0.7077
Epoch 63/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6567 - accuracy: 0.6746 - sensitivity: 0.6461 - specificity: 0.7081 - val_loss: 0.6702 - val_accuracy: 0.6450 - val_sensitivity: 0.6224 - val_specificity: 0.7077
Epoch 64/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6601 - accuracy: 0.6582 - sensitivity: 0.6234 - specificity: 0.6950 - val_loss: 0.6700 - val_accuracy: 0.6400 - val_sensitivity: 0.6224 - val_specificity: 0.6991
Epoch 65/100
10/10 [==============================] - 0s 4ms/

10/10 [==============================] - 0s 4ms/step - loss: 0.6884 - accuracy: 0.5925 - sensitivity: 0.5455 - specificity: 0.6304 - val_loss: 0.7081 - val_accuracy: 0.5250 - val_sensitivity: 0.6919 - val_specificity: 0.3829
Epoch 30/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6872 - accuracy: 0.6058 - sensitivity: 0.5679 - specificity: 0.6407 - val_loss: 0.7080 - val_accuracy: 0.5300 - val_sensitivity: 0.6919 - val_specificity: 0.3899
Epoch 31/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6817 - accuracy: 0.5995 - sensitivity: 0.5955 - specificity: 0.6026 - val_loss: 0.7076 - val_accuracy: 0.5500 - val_sensitivity: 0.6919 - val_specificity: 0.4514
Epoch 32/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6900 - accuracy: 0.6048 - sensitivity: 0.5294 - specificity: 0.6846 - val_loss: 0.7073 - val_accuracy: 0.5600 - val_sensitivity: 0.6836 - val_specificity: 0.5010
Epoch 33/100
10/10 [==============================] - 0s 4ms/

10/10 [==============================] - 0s 4ms/step - loss: 0.6834 - accuracy: 0.6030 - sensitivity: 0.4926 - specificity: 0.7066 - val_loss: 0.6894 - val_accuracy: 0.5600 - val_sensitivity: 0.6310 - val_specificity: 0.6336
Epoch 98/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6723 - accuracy: 0.6158 - sensitivity: 0.5345 - specificity: 0.6925 - val_loss: 0.6894 - val_accuracy: 0.5550 - val_sensitivity: 0.6368 - val_specificity: 0.6189
Epoch 99/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6665 - accuracy: 0.6356 - sensitivity: 0.5457 - specificity: 0.7044 - val_loss: 0.6895 - val_accuracy: 0.5600 - val_sensitivity: 0.6368 - val_specificity: 0.6262
Epoch 100/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6762 - accuracy: 0.6050 - sensitivity: 0.5000 - specificity: 0.6931 - val_loss: 0.6895 - val_accuracy: 0.5700 - val_sensitivity: 0.6368 - val_specificity: 0.6405
(None, 1)
Epoch 1/100
10/10 [==============================]

10/10 [==============================] - 0s 4ms/step - loss: 0.6601 - accuracy: 0.6350 - sensitivity: 0.6900 - specificity: 0.5840 - val_loss: 0.6943 - val_accuracy: 0.5600 - val_sensitivity: 0.4776 - val_specificity: 0.6199
Epoch 66/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6603 - accuracy: 0.6080 - sensitivity: 0.6170 - specificity: 0.5994 - val_loss: 0.6943 - val_accuracy: 0.5650 - val_sensitivity: 0.4918 - val_specificity: 0.6116
Epoch 67/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6606 - accuracy: 0.6172 - sensitivity: 0.6556 - specificity: 0.5825 - val_loss: 0.6941 - val_accuracy: 0.5600 - val_sensitivity: 0.4850 - val_specificity: 0.6116
Epoch 68/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6645 - accuracy: 0.6141 - sensitivity: 0.6517 - specificity: 0.5809 - val_loss: 0.6942 - val_accuracy: 0.5600 - val_sensitivity: 0.4850 - val_specificity: 0.6116
Epoch 69/100
10/10 [==============================] - 0s 4ms/

10/10 [==============================] - 0s 4ms/step - loss: 0.7095 - accuracy: 0.6261 - sensitivity: 0.6200 - specificity: 0.6355 - val_loss: 0.7172 - val_accuracy: 0.5850 - val_sensitivity: 0.7027 - val_specificity: 0.5585
Epoch 34/100
10/10 [==============================] - 0s 4ms/step - loss: 0.7062 - accuracy: 0.6253 - sensitivity: 0.6272 - specificity: 0.6224 - val_loss: 0.7168 - val_accuracy: 0.5800 - val_sensitivity: 0.7027 - val_specificity: 0.5085
Epoch 35/100
10/10 [==============================] - 0s 4ms/step - loss: 0.7094 - accuracy: 0.6183 - sensitivity: 0.6179 - specificity: 0.6206 - val_loss: 0.7151 - val_accuracy: 0.5800 - val_sensitivity: 0.7027 - val_specificity: 0.5085
Epoch 36/100
10/10 [==============================] - 0s 4ms/step - loss: 0.6928 - accuracy: 0.6445 - sensitivity: 0.6509 - specificity: 0.6426 - val_loss: 0.7152 - val_accuracy: 0.5700 - val_sensitivity: 0.6955 - val_specificity: 0.4999
Epoch 37/100
10/10 [==============================] - 0s 4ms/

In [ ]:
#@title Locally-Directed1D layer
"""Locally-Directed1D layer.
"""

@tf_export('keras.layers.LocallyDirected1D')
class LocallyDirected1D(Layer):
    """Locally-Directed1D layer for 1D inputs.

  The `LocallyDirected1D` layer works similarly to
  the `Conv1D` layer, except that weights are unshared,
  that is, a different set of filters is applied at each different patch
  of the input.

  Example:
  ```python
      # apply a unshared weight convolution 1d of length 3 to a sequence with
      # 10 timesteps, with 64 output filters
      model = Sequential()
      model.add(LocallyDirected1D(64, 3, input_shape=(10, 32)))
      # now model.output_shape == (None, 8, 64)
      # add a new conv1d on top
      model.add(LocallyDirected1D(32, 3))
      # now model.output_shape == (None, 6, 32)
  ```

  Arguments:
      mask: sparse matrix with shape (input, output) connectivity matrix,
            True defines connection between (in_i, out_j), should be sparse (False,0) >> True
            should be scipy sparese matrix in COO Format!
      filters: Integer, the dimensionality of the output space
          (i.e. the number of output filters in the convolution).
      kernel_size: An integer or tuple/list of a single integer,
          specifying the length of the 1D convolution window.
      strides: An integer or tuple/list of a single integer,
          specifying the stride length of the convolution.
          Specifying any stride value != 1 is incompatible with specifying
          any `dilation_rate` value != 1.
      padding: Currently only supports `"valid"` (case-insensitive).
          `"same"` may be supported in the future.
      data_format: A string,
          one of `channels_last` (default) or `channels_first`.
          The ordering of the dimensions in the inputs.
          `channels_last` corresponds to inputs with shape
          `(batch, length, channels)` while `channels_first`
          corresponds to inputs with shape
          `(batch, channels, length)`.
          It defaults to the `image_data_format` value found in your
          Keras config file at `~/.keras/keras.json`.
          If you never set it, then it will be "channels_last".
      activation: Activation function to use.
          If you don't specify anything, no activation is applied
          (ie. "linear" activation: `a(x) = x`).
      use_bias: Boolean, whether the layer uses a bias vector.
      kernel_initializer: Initializer for the `kernel` weights matrix.
      bias_initializer: Initializer for the bias vector.
      kernel_regularizer: Regularizer function applied to
          the `kernel` weights matrix.
      bias_regularizer: Regularizer function applied to the bias vector.
      activity_regularizer: Regularizer function applied to
          the output of the layer (its "activation")..
      kernel_constraint: Constraint function applied to the kernel matrix.
      bias_constraint: Constraint function applied to the bias vector.

  Input shape:
      3D tensor with shape: `(batch_size, steps, input_dim)`

  Output shape:
      3D tensor with shape: `(batch_size, new_steps, filters)`
      `steps` value might have changed due to padding or strides.
  """

    def __init__(self,
                 mask,
                 filters,
                 padding='valid',
                 data_format=None,
                 activation=None,
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 **kwargs):
        super(LocallyDirected1D, self).__init__(**kwargs)
        self.filters = filters
        self.padding = conv_utils.normalize_padding(padding)
        self.data_format = conv_utils.normalize_data_format(data_format)
        self.activation = activations.get(activation)
        self.use_bias = use_bias
        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)
        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.input_spec = InputSpec(ndim=3)
        self.mask = mask

    @tf_utils.shape_type_conversion
    def build(self, input_shape):
        if self.data_format == 'channels_first':
            input_dim, input_length = input_shape[1], input_shape[2]
        else:
            input_dim, input_length = input_shape[2], input_shape[1]

        if input_dim is None:
            raise ValueError('Axis 2 of input should be fully-defined. '
                             'Found shape:', input_shape)
        self.output_length = self.mask.shape[1]
        if self.data_format == 'channels_first':
            self.kernel_shape = (input_dim, input_length,
                                 self.filters, self.output_length)
        else:
            self.kernel_shape = (input_length, input_dim,
                                 self.output_length, self.filters)


        self.kernel = self.add_weight(shape=(len(self.mask.data),),    #sum of all nonzero values in mask sum(sum(mask))
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        self.kernel_idx = sorted(get_idx(self.mask))


        if self.use_bias:
            self.bias = self.add_weight(
                shape=(self.output_length, self.filters),
                initializer=self.bias_initializer,
                name='bias',
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint)
        else:
            self.bias = None

        if self.data_format == 'channels_first':
            self.input_spec = InputSpec(ndim=3, axes={1: input_dim})
        else:
            self.input_spec = InputSpec(ndim=3, axes={-1: input_dim})
        self.built = True


    def call(self, inputs):

        # output = local_conv_matmul(inputs, self.kernel_mask,
        #                            self.output_length)

        output = local_conv_matmul_sparse(inputs, self.mask, self.kernel, self.kernel_idx, self.output_length, self.filters)



        if self.use_bias:
            output = Kb.bias_add(output, self.bias, data_format=self.data_format)

        output = self.activation(output)
        return output

    # def get_config(self):  # delete this?
    #     config = {
    #         'filters':
    #             self.filters,
    #         'padding':
    #             self.padding,
    #         'data_format':
    #             self.data_format,
    #         'activation':
    #             activations.serialize(self.activation),
    #         'use_bias':
    #             self.use_bias,
    #         'kernel_initializer':
    #             initializers.serialize(self.kernel_initializer),
    #         'bias_initializer':
    #             initializers.serialize(self.bias_initializer),
    #         'kernel_regularizer':
    #             regularizers.serialize(self.kernel_regularizer),
    #         'bias_regularizer':
    #             regularizers.serialize(self.bias_regularizer),
    #         'activity_regularizer':
    #             regularizers.serialize(self.activity_regularizer),
    #         'kernel_constraint':
    #             constraints.serialize(self.kernel_constraint),
    #         'bias_constraint':
    #             constraints.serialize(self.bias_constraint),
    #     }
    #     base_config = super(LocallyDirected1D, self).get_config()
    #     return dict(list(base_config.items()) + list(config.items()))


def local_conv_matmul_sparse(inputs, mask, kernel, kernel_idx, output_length, filters):
    """Apply N-D convolution with un-shared weights using a single matmul call.

  Arguments:
      inputs: (N+2)-D tensor with shape
          `(batch_size, channels_in, d_in1, ..., d_inN)`
          or
          `(batch_size, d_in1, ..., d_inN, channels_in)`.
      mask: sparse matrix COO format connectivity matrix, shape: (input layer, output layer)
      kernel_idx =  the transposed mask matrix indices
      kernel: the unshared weights for N-D convolution,
          an (N+2)-D tensor of shape:
          `(d_in1, ..., d_inN, channels_in, d_out2, ..., d_outN, channels_out)`
          or
          `(channels_in, d_in1, ..., d_inN, channels_out, d_out2, ..., d_outN)`,
          with the ordering of channels and spatial dimensions matching
          that of the input.
          Each entry is the weight between a particular input and
          output location, similarly to a fully-connected weight matrix.
      output_shape: (mask.shape[1], mask.shape[0]) is used instead.
      output_length = length of the output
      filters =  standard 1

  Returns:
      Output (N+2)-D tensor with shape `output_shape`.
  """
    output_shape = (mask.shape[1], mask.shape[0])
    inputs_flat = Kb.reshape(inputs, (Kb.shape(inputs)[0], -1)) 

    # print("kernel_idx", len(kernel_idx))
    # print("inputs", Kb.shape(inputs_flat))
    # print("kernel", Kb.shape(kernel))

    output_flat = Kb.sparse_ops.sparse_tensor_dense_mat_mul(
        kernel_idx, kernel, (mask.shape[1], mask.shape[0]), inputs_flat, adjoint_b=True)
    # tf.print(kernel)
    
    output_flat_transpose= Kb.transpose(output_flat)
    output_reshaped = Kb.reshape(output_flat_transpose, [-1, output_length, filters])
    # output_reshaped = Kb.reshape(output_flat_transpose,[Kb.shape(output_flat_transpose)[0],] + output_shape.as_list()[1:])
    return output_reshaped

def get_idx(mask):
  """"returns the transposed coordinates in tuple form:
   [(mask.col[0], mask,row[0])...[mask.col[n], mask.row[n])]"""
  coor_list = []
  for i, j in zip(mask.col, mask.row):
    coor_list.append((i,j))

  return coor_list








In [ ]:
def GenNet_proof_of_concept_network(inputsize, mask):
    ''' The function that creates the neural network. 
    The locallyDirected1D layer let's you define all the connections yourself 
    with the help of a mask, the rest is standard Keras
    The created network is plotted at the end of the notebook'''

    inputs_ = K.Input((inputsize,), name='inputs_')

    Input_layer = K.layers.Reshape(input_shape=(inputsize,), target_shape=(inputsize, 1))(inputs_)
    
    # next line we use the new Gennet layer (LocallyDirected1D) to define all the connections ourself
    Gene_layer = LocallyDirected1D(mask=mask, filters=1, input_shape=(inputsize, 1), name="gene_layer",)(Input_layer)
    Gene_layer = K.layers.Flatten()(Gene_layer)
    Gene_layer = K.layers.Activation("tanh", name="gene_activation")(Gene_layer) #gene layer

    BatchNorm = K.layers.BatchNormalization(center=False, scale=False, name="inter_out")(Gene_layer)
    
    Prediction_layer = K.layers.Dense(units=1,    kernel_regularizer=K.regularizers.l1(0.01) )(BatchNorm)
    Prediction_layer = K.layers.Activation("sigmoid", name="output_activation")(Prediction_layer)
    model = K.Model(inputs=inputs_, outputs=Prediction_layer)
    return model

def make_mask_gene_layer(inputsize):
    '''We create a simple mask for this network, the first 5 are connected to the 
    first neuron. SNPs 5 to 10 are connected to the second neuron etc. 
    We save it as a sparse matrix.
    The created network is plotted at the end of the notebook'''
    mask_d  = np.zeros((inputsize,10), np.bool)
    mask_d[0:5,0]= True
    mask_d[5:10,1]= True
    mask_d[10:20,2]=True
    mask_d[20:30,3]=True
    mask_d[30:50,4]=True
    mask_d[50:70,5]= True
    mask_d[70:80,6]= True
    mask_d[80:90,7]=True
    mask_d[90:95,8]=True
    mask_d[95:100,9]=True
    mask =  coo_matrix(mask_d)
    gene_end =[0,5,10,20,30,50,70,80,90,95,100]  
    return mask, gene_end


In [ ]:
# training_data_path = './rethink-prs-master/mb-mdr/reformatted-data/train/'
test_data_path = './rethink-prs-master/mb-mdr/reformatted-data/train/'
training_data = os.listdir(training_data_path)
genemask, gene_end = make_mask_gene_layer(inputsize)
for file in performance.index.tolist():

    train = pd.read_csv(training_data_path + file, sep='\t').astype(int)
    SNPs = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']
    xtrain, ytrain = train[SNPs].values, train['Class'].values
    test = pd.read_csv(test_data_path + file, sep='\t').astype(int)

    xtrain, ytrain = train[SNPs].iloc[:600].values, train['Class'].iloc[:600].values
    xval, yval = train[SNPs].iloc[600:].values, train['Class'].iloc[600:].values
    xtest, ytest = test[SNPs].values, test['Class'].values


    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
    
    model = GenNet_proof_of_concept_network(inputsize=int(inputsize), mask=genemask)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, 
                  metrics=["accuracy", sensitivity, specificity]) # compile the network
    earlystop = K.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001,
                                          patience=50, verbose=0, mode='auto',
                                          restore_best_weights=True) # we stop training after not improving after 50 epochs on the validation data
    model.fit(x=xtrain, y=ytrain, batch_size=batch_size, epochs=epochs, 
              verbose=1, callbacks=[earlystop], validation_data=(xval,yval), shuffle=True) # train the neural network
    
    
    ypred = model.predict(xtest)
    auroc = roc_auc_score(ytest, ypred)
    
    performance_all.loc[file, 'auroc_NN_random_causal'] = auroc

In [ ]:
performance_all.to_csv('./rethink-prs-master/performance_MRS_complete.csv')

In [ ]:
test_data_path = './rethink-prs-master/mb-mdr/reformatted-data/train/'
performance_table = pd.read_csv('./performance.csv')
training_data = os.listdir(training_data_path)
for file in performance.index.tolist():
    SNPs = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9']
    
    train = pd.read_csv(training_data_path + file, sep='\t').astype(int)
    xtrain, ytrain = train[SNPs].values, train['Class'].values
    
    xtrain, ytrain = train[SNPs].iloc[:600].values, train['Class'].iloc[:600].values
    xval, yval = train[SNPs].iloc[600:].values, train['Class'].iloc[600:].values
    
    test = pd.read_csv(test_data_path + file, sep='\t').astype(int)
    xtest, ytest = test[SNPs].values, test['Class'].values
    
    # snippet to train the model and predict
    
    # ypred = ...
    
    auroc = roc_auc_score(ytest, ypred)
    performance_all.loc[file, 'auroc_NN_random_causal'] = auroc
    
    